# Create Individual Deforestation Files From Hansen Dataset

The purpose of this notebook is two fold:
1. Extract pixels:Takes the 'lossyear' image from the Hansen et al. (2013) dataset and creates a tiff file deforestation_year for each year desired. Take the 'treecover2000' and filter for pixels greater than 10%. 
2. 'lossyear' and 'treecover2000' also need to be cropped so that pixels are only within the boundary of the active property of that year. 

# Import  libraries 

In [1]:
import os
import sys
from pathlib import Path
import rasterio
from rasterio.merge import merge
from rasterio.mask import mask
from rasterio.transform import from_origin
import geopandas as gpd
import pandas as pd
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt


# Import Constants

In [2]:

# Get the current working directory
current_dir = os.path.abspath('')

# Search for the 'constants.py' file starting from the current directory and moving up the hierarchy
project_root = current_dir
while not os.path.isfile(os.path.join(project_root, 'constants.py')):
    project_root = os.path.dirname(project_root)

# Add the project root to the Python path
sys.path.append(project_root)


In [3]:
from constants import HANSEN_LOSSYEAR_FILEPATH, STUDY_BOUNDARY_PATH, OUTPUT_PATH, HANSEN_TREECOVER_FILEPATH, DF_BY_YEAR_BINARY, TREECOVER_10_AND_ABOVE, HANSEN_TEN_MASK, DATA_PATH

In [4]:
study_boundary_gdf = gpd.read_file(STUDY_BOUNDARY_PATH)

'''rasterio.open() expects a single file path as a string, 
 pass the file path string directly. Since HANSEN_LOSSYEAR_FILEPATHS is a list, 
 you can access the file path string by indexing the list with [0].
 '''
# Reproject the study boundary to match the Hansen raster CRS
with rasterio.open(HANSEN_LOSSYEAR_FILEPATH) as src:
    hansen_crs = src.crs

# Reproject the study boundary GeoDataFrame to match the Hansen raster CRS    
study_boundary_gdf_reprojected = study_boundary_gdf.to_crs(hansen_crs)
# Get the geometry column from the reprojected GeoDataFrame
study_boundary_geom_reprojected = study_boundary_gdf_reprojected.geometry

In [5]:
with rasterio.open(HANSEN_LOSSYEAR_FILEPATH) as hansen:
    hansen_array = hansen.read(1)
    hansen_crs = hansen.crs
    out_transform = hansen.transform
    out_meta = hansen.meta
    

In [6]:
with rasterio.open(HANSEN_LOSSYEAR_FILEPATH) as hansen:
    hansen_data = hansen.read(1)  # Reading the first band
    hansen_transform = hansen.transform
    hansen_meta = hansen.meta
    hansen_nodata = hansen.nodata

# Extract Tree Cover >= 10%

To extract pixels with 10% and above tree cover and write the resulting raster:

1. Read the tree cover raster data.
2. Create a binary mask for pixels with tree cover equal to or greater than 10%.
3. Write the binary mask to a new raster file.

In [14]:
# Step 1: Read the tree cover raster data
with rasterio.open(HANSEN_TREECOVER_FILEPATH) as src:
    tree_cover_array = src.read(1)
    tree_cover_transform = src.transform
    # Update the profile with the new nodata value
    profile = src.profile
    profile.update(nodata=-1, dtype='int16')

# Step 2: Create a mask for pixels with tree cover equal to or greater than 10%
'''np.where() to create a new array, masked_array,
that contains the original tree cover values 
where the condition (tree cover >= 10%) is met and 0 for the other pixels. '''
tree_cover_threshold = 10
masked_array = np.where(tree_cover_array >= tree_cover_threshold, tree_cover_array, -1)

# Step 3: Write the masked array to a new raster file
output_dir = os.path.join(OUTPUT_PATH, 'tree_cover_10_percent_and_above')
os.makedirs(output_dir, exist_ok=True)

output_filepath = os.path.join(output_dir, 'tree_cover_10_percent_and_above.tif')
with rasterio.open(output_filepath, 'w', **profile) as dst:
    dst.write(masked_array, 1)
    dst.transform = tree_cover_transform

In [11]:
profile

{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -1, 'width': 20381, 'height': 22512, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00026949458523585647, 0.0, -62.64186038139295,
       0.0, -0.00026949458523585647, -19.287457970745013), 'blockxsize': 256, 'blockysize': 256, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}

# Mask Hansen Lossyear with Treecover >= 10%

In [ ]:
# Load the Hansen tree cover raster
with rasterio.open(HANSEN_LOSSYEAR_FILEPATH) as hansen:
    hansen_data = hansen.read(1)  # Reading the first band
    hansen_transform = hansen.transform
    hansen_meta = hansen.meta
    hansen_nodata = hansen.nodata  # Read the nodata value

    # Convert hansen_data to a signed type to accommodate -1 values
    hansen_data = hansen_data.astype('int16')

    # Load the tree cover threshold raster
    with rasterio.open(TREECOVER_10_AND_ABOVE) as treecover10:
        tree_10_data = treecover10.read(1)  # Reading the first band
        tree_10_nodata = treecover10.nodata  # Read the nodata value

        # Convert tree_10_data to the same type as hansen_data
        tree_10_data = tree_10_data.astype('int16')

        # Mask where tree_10_data is 0 or nodata, and also consider nodata from hansen_data
        masked_tree_cover = np.where(
            (tree_10_data == 0) | (tree_10_data == tree_10_nodata) | (hansen_data == hansen_nodata),
            -1,  # New nodata value
            hansen_data  # Keep original hansen_data where conditions are not met
        )

    # Update the metadata for output
    hansen_meta.update(dtype='int16', nodata=-1)

    # Save the masked tree cover raster
    output_dir = os.path.join(OUTPUT_PATH, 'hansen_processed')
    os.makedirs(output_dir, exist_ok=True)

    hansen_masked_tree_cover_filepath = os.path.join(output_dir, "hansen_22_masked_10_above.tif")

    with rasterio.open(hansen_masked_tree_cover_filepath, "w", **hansen_meta) as dst:
        dst.write(masked_tree_cover, 1)  # Write the masked data to the first band


In [5]:
output_dir = os.path.join(DATA_PATH, 'raw-hansen')
os.makedirs(output_dir, exist_ok=True)
# Load the raster
with rasterio.open(HANSEN_TEN_MASK) as src:
    # Crop the raster with the shapefile
    out_image, out_transform = mask(src, study_boundary_geom_reprojected, crop=False)
    out_meta = src.meta.copy()

    # Update the metadata to reflect the new shape (height, width), transform, and nodata value
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform,
                     "nodata": -1})

    # Save the cropped raster
    output_raster_path = os.path.join(output_dir, "cropped__study_hansen_ten_mask.tif")
    with rasterio.open(output_raster_path, "w", **out_meta) as dest:
        dest.write(out_image)

# Extract pixels corresponding to each year (2001-2022).

One-hot encoding is typically used for categorical variables, where there is no inherent ordering or numerical relationship between the categories. In your case, the year of deforestation does have an inherent ordering (i.e., 2007 comes before 2008) and possibly a numerical relationship (i.e., deforestation in earlier years might influence deforestation in later years), so treating it as a categorical variable might not be the best approach.

However, there is a nuance in your case. If a pixel gets deforested in a particular year, it remains deforested in all subsequent years. This means that a pixel deforested in 2007 will also be deforested in 2008, 2009, and so on. If you use the year of deforestation as a numeric value, you might be inadvertently suggesting that deforestation in 2008 is "more" or "greater" than deforestation in 2007, which is not necessarily the case. Instead, you're just tracking the first year of deforestation for each pixel.

Given this, you might consider the following approach:

Binary Encoding: For each year, create a binary raster indicating whether deforestation occurred in that year or not. This would give you a series of rasters with values of [0, 1] for each year.

Stacking: Stack these rasters together to create a 3D array (height x width x time), where each layer in the third dimension corresponds to a year. This way, the model can learn about the temporal dynamics of deforestation.

This approach allows the model to learn from the temporal sequence of deforestation events without misinterpreting the year of deforestation as a numerical value. However, it does not explicitly encode the information that once a pixel is deforested, it remains deforested in all subsequent years. You will need to ensure that your model architecture can capture this temporal dependency.

In the end, the choice of encoding often depends on the specifics of your data and the model you're using, and it may be worth experimenting with different approaches to see what works best.


Lagged features are created by shifting the original feature values along the time dimension. They represent the values of a feature at an earlier time step. In the context of your deforestation problem, a lagged feature would represent whether deforestation occurred in a pixel in the previous year(s). By including these lagged features in your model, you allow the model to learn how deforestation in earlier years affects deforestation in later years.

Here's a step-by-step example of creating lagged features for your deforestation data:

Assume you have binary rasters for each year from 2001 to 2011, indicating whether deforestation occurred in that year (1) or not (0). You should have 11 rasters, one for each year.

To create a lagged feature with a lag of 1 year, you would shift the rasters by 1 year in the time dimension. For example, the 2002 raster would be shifted to 2001, the 2003 raster to 2002, and so on. The last raster (2011) would need to be filled with zeros or dropped, as there is no data available for 2012.

You should now have 11 rasters, where each raster represents whether deforestation occurred in the previous year (1) or not (0). These are the lagged features with a lag of 1 year.

Stack these new lagged rasters together with the original rasters. This would create a 3D array (height x width x 2 * time), where each layer in the third dimension corresponds to a year and its corresponding lagged year.

Use this new 3D array as input to your Random Forest model. The model will now learn the relationship between deforestation in the previous year(s) and the current year, which may help it better capture temporal dependencies.

In this example, I used a lag of 1 year, but you can experiment with different lag values depending on your data and problem. For instance, you could create lagged features for 2 or 3 years to capture longer-term dependencies.

Note that this is just one way to introduce temporal information into a non-temporal model like Random Forest. There might be other ways to incorporate temporal information, and it's worth experimenting to find the best approach for your data and problem.

In [ ]:
masked_tree = rasterio.open(HANSEN_TEN_MASK)

In [ ]:
masked_tree_cover = masked_tree.read(1)

In [7]:
'''This function accepts raster data, a start year, and an end year as arguments. 
The function iterates through the years within the given range and 
creates a binary mask for each year where the raster values match the year. 
It then stores these binary masks in a dictionary with the corresponding year 
as the key.'''



# Extract pixels corresponding to each year (2001-2021)
# This will return binary but if we want to keep the encoded value of the 
# pixelReturns pixels encoded with value of 1 and zeros as NaN.
#if `year_pixels[year_pixels == 0] = np.nan` is removed then will return 
# Unique values for year 2011: [0 1]. 

def extract_pixels_by_year_binary(raster_data, start_year, end_year):
    year_data = {}
    for year in range(start_year, end_year + 1):
        year_pixels = (raster_data == year).astype(int) 
        year_data[year] = year_pixels

        # Print unique values for each year
        unique_values = np.unique(year_pixels)
        print(f"Unique values for year {year + 2000}: {unique_values}") # Add 2000 to the year to get the correct year values
    return year_data

pixels_by_year = extract_pixels_by_year_binary(out_image, 0, 22)

'''While iterating through the years, the function also prints unique values for 
each year's binary mask. These unique values should be either 0 or 1, 
where 1 represents the pixels that have a deforestation event 
for that specific year, and 0 represents the pixels that do not.'''




Unique values for year 2000: [0 1]
Unique values for year 2001: [0 1]
Unique values for year 2002: [0 1]
Unique values for year 2003: [0 1]
Unique values for year 2004: [0 1]
Unique values for year 2005: [0 1]
Unique values for year 2006: [0 1]
Unique values for year 2007: [0 1]
Unique values for year 2008: [0 1]
Unique values for year 2009: [0 1]
Unique values for year 2010: [0 1]
Unique values for year 2011: [0 1]
Unique values for year 2012: [0 1]
Unique values for year 2013: [0 1]
Unique values for year 2014: [0 1]
Unique values for year 2015: [0 1]
Unique values for year 2016: [0 1]
Unique values for year 2017: [0 1]
Unique values for year 2018: [0 1]
Unique values for year 2019: [0 1]
Unique values for year 2020: [0 1]
Unique values for year 2021: [0 1]
Unique values for year 2022: [0 1]


"While iterating through the years, the function also prints unique values for \neach year's binary mask. These unique values should be either 0 or 1, \nwhere 1 represents the pixels that have a deforestation event \nfor that specific year, and 0 represents the pixels that do not."

Returns pixels encoded with value of corresponding year(11,12,13...) and zeros as NaN.
if `year_pixels[year_pixels == 0] = np.nan` is removed then will return [0 11]. 

In [ ]:

def extract_pixels_by_year(raster_data, start_year, end_year):
    year_data = {}
    for year in range(start_year, end_year + 1):
        year_pixels = (raster_data == year).astype(int) * year
        year_data[year] = year_pixels

        # Print unique values for each year
        unique_values = np.unique(year_pixels)
        print(f"Unique values for year {year + 2000}: {unique_values}") # Add 2000 to the year to get the correct year values
             
    return year_data

pixels_by_year = extract_pixels_by_year(masked_tree_cover, 1, 22)



In [ ]:
'''cumulative binary mask for each year, where the mask indicates which pixels match the current year 
or any previous year within the given range. pixels had an event in that year or any year before it.'''
'''def extract_pixels_by_year_cumulative(raster_data, start_year, end_year):
    year_data = {}
    cumulative_pixels = np.zeros_like(raster_data, dtype=int)
    
    for year in range(start_year, end_year + 1):
        year_pixels = (raster_data == year).astype(int)
        cumulative_pixels += year_pixels
        year_data[year] = cumulative_pixels.copy()

        # Print unique values for each year
        unique_values = np.unique(year_pixels)
        print(f"Unique values for year {year + 2000}: {unique_values}") # Add 2000 to the year to get the correct year values
    return year_data

pixels_by_year = extract_pixels_by_year_cumulative(hansen_array, 1, 22)'''


In [ ]:
def extract_10year_deforestation(raster_data, start_year):
    cumulative_pixels = np.zeros_like(raster_data, dtype=int)
    
    # Loop through the 10-year interval
    for year in range(start_year, start_year + 10):
        year_pixels = (raster_data == year).astype(int)
        cumulative_pixels += year_pixels

    return cumulative_pixels

# Create a dictionary to store the 10-year datasets
ten_year_datasets = {}

# Loop through the years to generate the 10-year datasets
for start_year in range(1, 14):  # 1-10, 2-11, ... , 13-22
    ten_year_datasets[start_year] = extract_10year_deforestation(hansen_array, start_year)


# Write raster files for each year 
write_year_rasters() takes four arguments: year_data, out_transform, out_meta, and output_dir. The year_data is a dictionary containing binary masks for each year, out_transform and out_meta are the transform and metadata extracted from the original Hansen loss year raster, and output_dir is the directory where the output raster files should be saved.

Inside the function,  loop through the year_data dictionary, and for each year,  create an output file path with a filename based on the year (e.g., 'deforestation_2001.tif'). Then, use rasterio.open() in write mode ('w') to create a new raster file with the specified metadata. write the binary mask data to the raster and set its transform to match the original raster.

After defining the function, create an output directory deforestation_by_year inside the OUTPUT_PATH directory using os.makedirs(). The exist_ok=True parameter ensures that the function does not raise an error if the directory already exists.

Finally, call the write_year_rasters() function, passing the pixels_by_year dictionary, out_transform, out_meta, and output_dir. The function writes separate raster files for each year's binary mask in the specified output directory.


In [10]:
# Write raster files for each year
def write_year_rasters(year_data, out_transform, out_meta, output_dir):
    for year, data in year_data.items():
        out_filepath = os.path.join(output_dir, f'deforestation_{year}.tif')
        with rasterio.open(out_filepath, 'w', **out_meta) as dst:
            dst.write(data)
            dst.transform = out_transform

output_dir = os.path.join(OUTPUT_PATH, 'deforestation_by_year_binary')
os.makedirs(output_dir, exist_ok=True)

write_year_rasters(pixels_by_year, out_transform, out_meta, output_dir)

In [ ]:
# Write raster files for each year
def write_year_rasters(year_data, out_transform, out_meta, output_dir):
    for year, data in year_data.items():
        out_filepath = os.path.join(output_dir, f'deforestation_{year}.tif')
        with rasterio.open(out_filepath, 'w', **out_meta) as dst:
            dst.write(data, 1)
            dst.transform = out_transform

output_dir = os.path.join(OUTPUT_PATH, 'deforestation_by_year')
os.makedirs(output_dir, exist_ok=True)

write_year_rasters(pixels_by_year, hansen_transform, hansen_meta, output_dir)


In [ ]:
# Write raster files for 10-year intervals
def write_10year_rasters(year_data, out_transform, out_meta, output_dir):
    for start_year, data in year_data.items():
        # Adjust the naming convention to reflect the 10-year interval
        end_year = start_year + 9
        out_filepath = os.path.join(output_dir, f'deforestation_{start_year}-{end_year}.tif')
        with rasterio.open(out_filepath, 'w', **out_meta) as dst:
            dst.write(data, 1)
            dst.transform = out_transform

# Define the output directory
output_dir = os.path.join(OUTPUT_PATH, 'deforestation_10year_intervals')
os.makedirs(output_dir, exist_ok=True)

# Call the adjusted function
write_10year_rasters(ten_year_datasets, out_transform, out_meta, output_dir)


In [ ]:
# Read one of the TIF files and print its unique values
with rasterio.open("") as src:
    data = src.read(1)
print("Unique values in uncropped TIF:", np.unique(data))

# Creating Treecover for each year


In [14]:
#backwards to what i need
'''# Load the tree cover for the year 2000
with rasterio.open(TREECOVER_10_AND_ABOVE) as src:
    tree_cover_2000 = src.read(1)  # Assuming tree cover is in the first band
    tree_cover_meta = src.meta.copy()

    # Set a nodata value in the metadata if not already defined
    if 'nodata' not in tree_cover_meta or tree_cover_meta['nodata'] is None:
        tree_cover_meta['nodata'] = 0  # Assuming 0 represents areas without trees

# Load the deforestation data
with rasterio.open("C:/Users/bsf31/Documents/post-meds/data/policy-data/raw-hansen/cropped__study_hansen_ten_mask.tif") as src:
    deforestation_data = src.read(1)  # Assuming deforestation data is in the first band

# Iterate through each year from 2001 to 2022
for year in range(2000, 2023):
    # Create a copy of the baseline tree cover for modification
    updated_tree_cover = tree_cover_2000.copy()

    # Define the deforestation year relative to the dataset's scale
    deforestation_year = year - 2000  # Convert to deforestation data scale

    # Update tree cover based on deforestation data
    # Only update pixels that are not nodata (i.e., have tree cover probability >= 10)
    tree_cover_mask = updated_tree_cover >= 10  # Tree cover probability >= 10
    deforestation_mask = (deforestation_data <= deforestation_year) & tree_cover_mask
    updated_tree_cover[deforestation_mask] = tree_cover_meta['nodata']  # Set to nodata where deforestation occurred

    # Save the updated tree cover as a new file
    output_path = f'updated_tree_cover_{year}.tif'
    with rasterio.open(output_path, 'w', **tree_cover_meta) as dest:
        dest.write(updated_tree_cover, 1)'''

In [21]:
# Load the tree cover for the year 2000
with rasterio.open(TREECOVER_10_AND_ABOVE) as src:
    tree_cover_base = src.read(1)  # Assuming tree cover is in the first band
    tree_cover_meta = src.meta.copy()

    # Set a nodata value in the metadata if not already defined
    tree_cover_meta['nodata'] = 0

# Load the deforestation data
with rasterio.open("C:/Users/bsf31/Documents/post-meds/data/policy-data/raw-hansen/cropped__study_hansen_ten_mask.tif") as src:
    deforestation_data = src.read(1)  # Assuming deforestation data is in the first band

# Iterate through each year from 2001 to 2022
for year in range(2001, 2023):
    # Instead of using the original base for each year, start with the updated tree cover from the previous year
    # For the first year (2001), this will be the base tree cover
    if year == 2001:
        updated_tree_cover = tree_cover_base.copy()
    else:
        # For subsequent years, load the updated tree cover from the previous year
        with rasterio.open(f'updated_tree_cover_{year - 1}.tif') as src:
            updated_tree_cover = src.read(1)

    # Define the deforestation year relative to the dataset's scale
    deforestation_year = year - 2000  # Convert to deforestation data scale

    # Identify deforested pixels for the current year
    deforested_mask = deforestation_data == deforestation_year

    # Set deforested pixels to nodata in the tree cover
    updated_tree_cover[deforested_mask] = tree_cover_meta['nodata']

    # Save the updated tree cover as a new file
    output_path = f'updated_tree_cover_{year}.tif'
    with rasterio.open(output_path, 'w', **tree_cover_meta) as dest:
        dest.write(updated_tree_cover, 1)


In [23]:
folder_path = r"C:\Users\bsf31\Documents\post-meds\data\policy-data\raw-hansen\tree_cover_10_percent_and_above - Copy"

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.tif'):  # Ensure we're only processing TIFF files
        file_path = os.path.join(folder_path, filename)
        
        # Open the existing tree cover file
        with rasterio.open(file_path) as src:
            data = src.read(1)  # Read the first band
            meta = src.meta.copy()
            
            # Ensure nodata is correctly set to 0
            meta['nodata'] = 0

            # Transform pixel values: > 0 to 1, others remain as nodata (0)
            data[data > 0] = 1

        # Overwrite the file with updated data
        with rasterio.open(file_path, 'w', **meta) as dest:
            dest.write(data, 1)

print("Processing complete.")


Processing complete.
